# YOLO Training - Bone Fracture Detection

## Module 3: Object Detection Training

Complete YOLOv8 training pipeline with 5-10 epochs on Google Colab GPU.


In [4]:
# Install dependencies
!pip install ultralytics opencv-python opencv-contrib-python scikit-image scipy -q
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118 -q

print("✅ Dependencies installed!")



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Dependencies installed!



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Check GPU
import torch
print("="*60)
print("GPU INFORMATION")
print("="*60)
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"VRAM: {vram_gb:.2f} GB")
    print("✅ GPU ready!")
    recommended_model = 's' if vram_gb >= 8 else 'n'
    recommended_batch = 16 if vram_gb >= 8 else 8
    print(f"Recommended: YOLOv8{recommended_model}, batch={recommended_batch}")
else:
    print("⚠️  No GPU! Go to: Runtime > Change runtime type > GPU")
print("="*60)


GPU INFORMATION
CUDA Available: False
⚠️  No GPU! Go to: Runtime > Change runtime type > GPU


In [ ]:
# Mount Google Drive & Setup
from google.colab import drive
from pathlib import Path
import os

drive.mount('/content/drive')

project_dir = Path("/content/bone_fracture_detection")
project_dir.mkdir(exist_ok=True)
(project_dir / "data").mkdir(exist_ok=True, parents=True)
(project_dir / "yolo_training_results").mkdir(exist_ok=True, parents=True)

print("✅ Setup complete!")
print("\n📁 Upload dataset:")
print("   1. Click folder icon (left sidebar)")
print("   2. Upload 'data/archive.zip'")
print("   3. Or: !cp -r /content/drive/MyDrive/bone_fracture_dataset /content/bone_fracture_detection/data/")


In [ ]:
# Extract dataset (if uploaded as zip)
# !unzip -q /content/archive.zip -d /content/bone_fracture_detection/data/

print("✅ Dataset ready!")


In [ ]:
# Training Configuration
DATA_YAML = "/content/bone_fracture_detection/data/archive/bone fracture detection.v4-v4.yolov8/data.yaml"
MODEL_SIZE = "n"  # 'n', 's', 'm', 'l', 'x'
EPOCHS = 10       # 5-10 epochs
BATCH = 16        # 8 (4GB), 16 (8GB+), 32 (16GB+)
IMGSZ = 640

print("="*80)
print("TRAINING CONFIGURATION")
print("="*80)
print(f"Model: YOLOv8{MODEL_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Batch: {BATCH}")
print(f"Image Size: {IMGSZ}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print("="*80)


In [ ]:
# Update Data YAML
import yaml

data_yaml_path = Path(DATA_YAML)
if data_yaml_path.exists():
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    base_dir = data_yaml_path.parent
    data_config['train'] = str(base_dir / 'train' / 'images')
    data_config['val'] = str(base_dir / 'valid' / 'images')
    data_config['test'] = str(base_dir / 'test' / 'images')
    
    updated_yaml = base_dir / 'data_updated.yaml'
    with open(updated_yaml, 'w') as f:
        yaml.dump(data_config, f)
    
    DATA_YAML = str(updated_yaml)
    print(f"✅ Data YAML updated: {updated_yaml}")
else:
    print(f"⚠️  Dataset not found: {data_yaml_path}")


In [ ]:
# Start Training
from ultralytics import YOLO

print("="*80)
print("STARTING YOLO TRAINING")
print("="*80)
print(f"This will take ~2-5 hours for {EPOCHS} epochs...")
print("="*80)

model = YOLO(f"yolov8{MODEL_SIZE}.pt")

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=str(project_dir / "yolo_training_results"),
    name=f"yolov8{MODEL_SIZE}_bone_fracture",
    save=True,
    save_period=2,
    val=True,
    plots=True,
    verbose=True,
    seed=42,
    amp=True
)

print("\n✅ Training Complete!")


In [ ]:
# View Results & Download Model
from IPython.display import Image, display
from google.colab import files

results_dir = project_dir / "yolo_training_results" / f"yolov8{MODEL_SIZE}_bone_fracture"
best_model = results_dir / "weights" / "best.pt"

if best_model.exists():
    print(f"✅ Best model: {best_model}")
    
    # Display results
    try:
        display(Image(f"{results_dir}/results.png"))
        display(Image(f"{results_dir}/confusion_matrix.png"))
    except:
        print(f"Results in: {results_dir}")
    
    # Download
    files.download(str(best_model))
    print("✅ Model downloaded!")
    
    # Save to Drive
    !cp {best_model} /content/drive/MyDrive/
    print("✅ Model saved to Drive!")
else:
    print("⚠️  Model not found")
